In [1]:
%load_ext autoreload
%autoreload 2

import gzip
import math
import pandas
import StringIO
import textwrap
import dendropy
from ngs_parsers import VCF
import pandas.rpy.common as com
import rpy2.robjects as robjects
from itertools import combinations
from collections import OrderedDict
from rpy2.robjects.packages import importr
import numpy as np

vcf = VCF.VCF('/Users/testudines/DATA/5M/VCF/RNAseq_sample_lab_ids_v2.vcf.gz')
vcf.populations  =   {'AGAW': ['ETAI014','ETAI015','ETAI018',
                               'ETAI020','ETAI021','ETAI023',
                               'ETAI026','ETAI028','ETAI029',
                               'ETAI030','ETAI032','ETAI033',
                               'ETAS003','ETAS024','ETAS026',
                               'ETAS030','ETAS031','ETAS034',
                               'ETAS035'],
                    'AMHARA': ['ETAM041','ETAM042','ETAM043',
                               'ETAM044','ETAM045','ETAM046',
                               'ETAM047','ETAM048','ETAM049',
                               'ETAM051','ETAM052','ETAM053',
                               'ETAM055','ETAM058','ETAM060',
                               'ETAM061','ETAM063','ETAM070',
                               'ETAM080'],
                   'ARGOBBA': ['ETAG001','ETAG002','ETAG004',
                               'ETAG007','ETAG008','ETAG009',
                               'ETAG013','ETAG014','ETAG015',
                               'ETAG017','ETAG018','ETAG019',
                               'ETAG022','ETAG023','ETAG027',
                               'ETAG029'],
                      'DIZI': ['ETDI001','ETDI002','ETDI006',
                               'ETDI008','ETDI011','ETDI013',
                               'ETDI014','ETDI015','ETDI019',
                               'ETDI020','ETDI021','ETDI022',
                               'ETDI023','ETDI027','ETDI028',
                               'ETDI036','ETDI037','ETDI039',
                               'ETDI040'],
                   'HADZABE': ['TZHZ401','TZHZ402','TZHZ442',
                               'TZHZ446','TZHZ457','TZHZ463',
                               'TZHZ467','TZHZ469','TZHZ476',
                               'TZHZ479','TZHZ488','TZHZ492',
                               'TZHZ497','TZHZ505','TZHZ506',
                               'TZHZ509','TZHZ533','TZHZ550',
                               'TZHZ555'],
                     'MURSI': ['ETMS002','ETMS004','ETMS006',
                               'ETMS008','ETMS009','ETMS011',
                               'ETMS012','ETMS013','ETMS014',
                               'ETMS018','ETMS020','ETMS021',
                               'ETMS022','ETMS023','ETMS027',
                               'ETMS029','ETMS033','ETMS034',
                               'ETMS036'],
                     'SABUE': ['ETSB001','ETSB002','ETSB004',
                               'ETSB005','ETSB006','ETSB008',
                               'ETSB009','ETSB012','ETSB013',
                               'ETSB014','ETSB016','ETSB023',
                               'ETSB027','ETSB029','ETSB031',
                               'ETSB035','ETSB036','ETSB039'],
                   'SANDAWE': ['TZSW140','TZSW142','TZSW145',
                               'TZSW156','TZSW161','TZSW162',
                               'TZSW174','TZSW177','TZSW180',
                               'TZSW182','TZSW188','TZSW193',
                               'TZSW197','TZSW205',
                               'TZSW209'],
                     'WEYTO': ['ETWE002','ETWE005','ETWE007',
                               'ETWE008','ETWE009','ETWE010',
                               'ETWE011','ETWE014','ETWE016',
                               'ETWE017','ETWE020','ETWE023',
                               'ETWE027','ETWE030','ETWE032',
                               'ETWE033','ETWE036','ETWE038']}

/Library/Python/2.7/site-packages/pandas/rpy/__init__.py:8: FutureWarning: The pandas.rpy module is deprecated and will be removed in a future version. We refer to external packages like rpy2, found here: http://rpy.sourceforge.net
  "like rpy2, found here: http://rpy.sourceforge.net", FutureWarning)


In [6]:
# def Wrights_Fst(ac, pop_ids):
#     """Does not correct for sample size."""
    
#     af = ac.ix[:,0].T[pop_ids] / ac.sum(axis=1).T[pop_ids]
#     af_dif = af.subtract(af.mean(axis=1), axis=0)
#     s2 = np.sum(af_dif.pow(2) / (len(pop_ids) -1), axis=1)
#     pm = af.mean(axis=1)
#     Fst = s2 / (pm * (1 - pm))
#     Fst = Fst.mean()
#     return Fst


# def Weirs_Fst(ac, h, ps, pop_ids):
    
#     n = ps[pop_ids].mean(axis=1)
#     r = float(len(pop_ids))
#     hm = h[pop_ids].mean(axis=1)

#     af = ac.ix[:,0].T[pop_ids] / ac.sum(axis=1).T[pop_ids] # calc allele freq (p)
#     pm = af.mean(axis=1)                                   # calc. p bar
#     af_dif = af.subtract(pm, axis=0)
#     s2 = np.sum(af_dif.pow(2) / (len(pop_ids) -1), axis=1)


#     N = s2 - (1 / (2*n-1)) * ((pm*(1 - pm))-(((r - 1)/r) * s2)-(hm/4))
#     D = (pm*(1 - pm)) + s2/r
#     Fst = N.sum() / D.sum()
#     return Fst


# # def subset_pops(dct, pops):
# #     return OrderedDict([ i for i in dct.iteritems() if i[0] in pops])
    

# def make_empty_df(pops):
#     data = np.zeros((len(pops),len(pops)))
#     df = pandas.DataFrame(data, index=pops, columns=pops)
#     return df
    

# def create_tables(vcf):

#     allele_counts = {}
#     hets = []
#     pop_sizes = []
#     snp_ids = []

#     for c, i in enumerate(vcf.vcf_file_iterator()):
#         if c > 25: break
        
#         ac = vcf.calc_allele_counts(i)

#         h  = vcf.calc_heterozygosity(i)
#         ps = vcf.get_population_sizes(i)

#         # snp_id = '{}_{}'.format(i['CHROM'],i['POS'])
#         snp_id = c
#         snp_ids.append(snp_id)
#         hets.append(h)
#         pop_sizes.append(ps)

#         allele_counts[snp_id] = pandas.DataFrame(ac)
    
    
#     ac = pandas.Panel(allele_counts)
#     h = pandas.DataFrame(hets, index=snp_ids)
#     ps = pandas.DataFrame(pop_sizes, index=snp_ids)
    
#     return ac, h, ps

# def mean_pairwise_Fst(pops, min_sample_count=7):
    
#     # create empty distance matrix
#     df = make_empty_df(pops)
    
#     ac, h, ps = create_tables(vcf)
#     print ac
#     for pop1, pop2 in combinations(pops,2):
        
#         Fst = Weirs_Fst(ac, h, ps, [pop1, pop2])
#         df.ix[pop1,pop2] = Fst
#         df.ix[pop2,pop1] = Fst

#     return df

        
# pops = vcf.populations.keys()
# df = mean_pairwise_Fst(pops)
# tree = calc_nj_tree(df)
# print tree


<class 'pandas.core.panel.Panel'>
Dimensions: 26 (items) x 4 (major_axis) x 9 (minor_axis)
Items axis: 0 to 25
Major_axis axis: 0 to 3
Minor_axis axis: AGAW to WEYTO
(DIZI:-0.0002468941188,(((AGAW:0.008242019559,ARGOBBA:-0.004578337855):0.003652023069,AMHARA:0.002076544695):0.01621587507,WEYTO:-0.005916750952):0.02473709469,((SABUE:0.002250552152,SANDAWE:-0.000655500069):0.005910848068,(MURSI:0.0153816623,HADZABE:0.03778400337):0.009288255912):0.005116271668);


In [2]:
import StringIO 
from dendropy import Tree

def make_empty_df(pops):
    data = np.zeros((len(pops),len(pops)))
    df = pandas.DataFrame(data, index=pops, columns=pops)
    return df

def extract_allele_counts(vcf):
    
    pops = []
    samples = []
    for p, spls in vcf.populations.iteritems():
        for s in spls:
            pops.append(p)
            samples.append(s)
    
    df =[]
    for c, i in enumerate(vcf.vcf_file_iterator()):
        #if c > 500: break
            
        ac = {}   
        for s in samples:

            if i[s] is None:
                ac[s] = None

            elif i[s]['GT'] == '0/0':
                ac[s] = 0

            elif i[s]['GT'] == '1/0' or i[s]['GT'] == '0/1':
                ac[s] = 1

            elif i[s]['GT'] == '1/1':
                ac[s] = 2

        df.append(ac)

    df = pandas.DataFrame(df)
    return df


def ac_2_af(df, pops):
    """Convert table of allele counts to table of allele frequencies."""  
    # Calculate Allele Frequencies  
    new_df = {pop: df[pops[pop]].sum(axis=1).divide(df[pops[pop]].shape[1]*2.0) for pop in pops}
    return pandas.DataFrame(new_df)


def Wrights_Fst(df, ps_dict, bootstrap=False):
    """Does not correct for sample size."""
    pop_ids = ps_dict.keys()
    af = ac_2_af(df, ps_dict, bootstrap)
    p = af.ix[:,pop_ids]
    p_bar = af.ix[:,pop_ids].mean(axis=1)
    p_dif = p.subtract(p_bar, axis=0)
    s2 = np.sum(p_dif.pow(2) / (len(pop_ids) -1), axis=1)
    Fst = s2 / (p_bar * (1 - p_bar))
    Fst = Fst.mean()
    if np.isinf(Fst) == True:
        Fst = 0
    return Fst


def Weirs_Fst(af, h, ps, ps_dict, bootstrap=False):
    
    pop_ids = ps_dict.keys()
    
    r = float(len(pop_ids))
    n_bar = ps[pop_ids].mean(axis=1)
    h_bar = h[pop_ids].mean(axis=1)
    p_bar = af.mean(axis=1)                                   # calc. p bar    
    
    af_dif = af.subtract(p_bar, axis=0)
    s2 = np.sum(af_dif.pow(2) / (len(pop_ids) -1), axis=1)

    N = s2 - (1 / (2*n_bar-1)) * (p_bar*(1 - p_bar)-(r - 1)/r * s2 - h_bar/4)
    D = (p_bar*(1 - p_bar)) + s2/r
    Fst = N.sum() / D.sum()
    return Fst


def ac_2_het(df, pops):
    ones = df==1
    new_df = {pop : ones[pops[pop]].sum(axis=1).divide(ones[pops[pop]].shape[1]) for pop in pops}
    return pandas.DataFrame(new_df)


def ac_2_ps(df, pops):
    new_df = {pop : df[pops[pop]].count(axis=1) for pop in pops}
    return pandas.DataFrame(new_df)   


def mean_pairwise_Fst(ac, pops):
    
    #pops = vcf.populations
    
    # create empty distance matrix
    final_df = make_empty_df(pops)
    
    h = ac_2_het(ac, pops)
    ps = ac_2_ps(ac, pops)
    
    for pop1, pop2 in combinations(pops,2):
        
        ps_dict = {k: pops[k] for k in (pop1, pop2)}
        af = ac_2_af(ac, ps_dict)
        
        Fst = Weirs_Fst(af, h, ps, ps_dict)
        
        final_df.ix[pop1,pop2] = Fst
        final_df.ix[pop2,pop1] = Fst

    return (final_df, h, ps, af)


def make_boot_rep(pops):
    # Bootstraping
    #   Resample with replacement each population.
    
    return {pop: list(np.random.choice(samples, len(samples), replace=True)) \
            for pop, samples in pops.iteritems()}


def calc_nj_tree(mtx):
    mtx = com.convert_to_r_matrix(mtx)
    ape = importr('ape', on_conflict='warn')
    as_dist = robjects.r['as.dist']
    NJ = robjects.r['nj']
    tree = NJ(as_dist(mtx))
    #pt = robjects.r['plot']
    #pt(tree)
    nwk = ape.write_tree(tree)[0]
    return nwk
    
pops = vcf.populations
ac = extract_allele_counts(vcf)
print 'allele counts done.'
final_df, h, ps, af =  mean_pairwise_Fst(ac, pops)
tree = calc_nj_tree(final_df)
print tree

# ntrees = []
# trees= []
# for i in range(0,500):

#     if i % 25 == 0:
#         print i
#     b_pops = make_boot_rep(pops)
#     #print b_pops
#     final_df, h, ps, af =  mean_pairwise_Fst(ac, b_pops)
#     tree = calc_nj_tree(final_df)
#     ntrees.append(tree)
#     tree = Tree.get_from_string(tree, schema="newick")
#     trees.append(tree)

# trees = dendropy.TreeList(trees)
# con_tree = trees.consensus(min_freq=0.50)
# con_tree.as_string('newick')

allele counts done.
(AGAW:0.007493583098,AMHARA:0.007838946313,((((((MURSI:0.01208370322,SABUE:0.04218702663):0.005896015748,HADZABE:0.04393485503):0.002859667951,SANDAWE:0.01477960955):0.007076610095,DIZI:0.01301411754):0.01362818158,WEYTO:0.021065968):0.003648512114,ARGOBBA:0.008202890016):0.0003616729256);


In [5]:
final_df

,MURSI,AMHARA,AGAW,SABUE,HADZABE,WEYTO,SANDAWE,DIZI,ARGOBBA
MURSI,0.000000,0.054183,0.053368,0.054271,0.062895,0.061420,0.039172,0.038306,0.051566
AMHARA,0.054183,0.000000,0.015333,0.086608,0.081127,0.032534,0.046172,0.038821,0.016424
AGAW,0.053368,0.015333,0.000000,0.086233,0.080721,0.032102,0.045626,0.038988,0.016038
SABUE,0.054271,0.086608,0.086233,0.000000,0.091037,0.093310,0.070510,0.059145,0.084790
HADZABE,0.062895,0.081127,0.080721,0.091037,0.000000,0.090051,0.057404,0.065630,0.079795
WEYTO,0.061420,0.032534,0.032102,0.093310,0.090051,0.000000,0.056742,0.047315,0.033766
SANDAWE,0.039172,0.046172,0.045626,0.070510,0.057404,0.056742,0.000000,0.039126,0.045412
DIZI,0.038306,0.038821,0.038988,0.059145,0.065630,0.047315,0.039126,0.000000,0.037714
ARGOBBA,0.051566,0.016424,0.016038,0.084790,0.079795,0.033766,0.045412,0.037714,0.000000


In [4]:
con_tree = trees.consensus(min_freq=0.20)
con_tree.as_string('newick')

'[&U] (ARGOBBA,(WEYTO:0.0343027933097,(DIZI:0.0255149399844,(SANDAWE:0.0301677678063,(HADZABE:0.0558620214162,(SABUE:0.0547466835168,MURSI:0.0249934620731)1.0000:0.00601069306252)0.8380:0.00287351697484)1.0000:0.00667493249322)1.0000:0.0134846993311)0.9960:0.00349287435667,(AMHARA:0.0204171210698,AGAW:0.020374795018)0.5260:0.00089361577623);\n'

In [28]:
mtx = com.convert_to_r_matrix(final_df)
ape = importr('ape', on_conflict='warn')
as_dist = robjects.r['as.dist']
NJ = robjects.r['nj']
tree = NJ(as_dist(mtx)) #,"unrooted")

In [38]:
for t in ntrees:
    #print t
    pass

In [9]:
ac = extract_allele_counts(vcf)
final_df, h, ps, af =  mean_pairwise_Fst(ac, pops)
tree = calc_nj_tree(final_df)
tree

KeyboardInterrupt: 